# Phase 1: Indexing
In this notebook:
- Import libraries, load configuration variables and create clients
- Indexing functions: create the index, chunk rows/documents and index chunks in AI Search
- Index data from a database: retrieve data from a Database with sql query, chunk the content and index the chunks in AI Search
    + Index data from a PostgreSQL database
    + Index data from a database using an endpoint (SQLite + Flask endpoint)
- Index contents of files: convert PDF files to markdown, chunk and index in AI Search
    + Index contents of files in a local folder
    + Index contents of files in a blob storage

## Import libraries, load configuration variables and create clients

In [ ]:
#%pip install azure-ai-documentintelligence
#%pip install langchain
#%pip install python-dotenv
#%pip install tiktoken
#%pip install openai
#%pip install azure-search-documents
#%pip install pg8000

In [ ]:
# Import libraries
import os
import sys
import requests
import json
import time
import pandas as pd

from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import DocumentContentFormat
from langchain.text_splitter import TokenTextSplitter

sys.path.append(os.path.abspath('..'))
from common_utils import *

# Load Azure OpenAI and AI Search variables and create clients
openai_config, ai_search_config = load_config()

# Load Document Intelligence configuration
doc_intel_endpoint = os.getenv("DOC_INTEL_ENDPOINT")
doc_intel_key = os.getenv("DOC_INTEL_KEY")
doc_intel_client = DocumentIntelligenceClient(endpoint=doc_intel_endpoint, credential=AzureKeyCredential(doc_intel_key))
print(f'doc_intel_endpoint: {doc_intel_endpoint}')

# Load SQLite endpoint (run server with 'python app.py')
sqlite_endpoint = os.environ["SQLITE_ENDPOINT"]
sqlite_user = os.environ["SQLITE_USER"]
sqlite_password = os.environ["SQLITE_PASSWORD"]
print(f'sqlite_endpoint: {sqlite_endpoint}')

MAX_TOKENS = 512
OVERLAP_TOKENS = 128 # 25% of 512 tokens is 128 tokens

## Indexing functions
Personalization and details:
- **create_index:** specify your keyword fields and your embeddings fields
- **index_documents:** the parameter 'content' is a list in json format with the fields defined when creating the index, converting the data from your source to that json list

In [25]:
# Create AI Search index
def create_index(index_name):
    # Create an Azure AI Search index client
    index_client = SearchIndexClient(endpoint=ai_search_config["ai_search_endpoint"], credential=ai_search_config["ai_search_credential"])
    
    # Fields definition
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
        SearchableField(name="title", type=SearchFieldDataType.String), #analyzer="es.microsoft"),
        SearchableField(name="content", type=SearchFieldDataType.String), #analyzer="es.microsoft"),
        SearchField(name="embeddingTitle", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True, vector_search_dimensions=EMBEDDINGS_DIMENSIONS, vector_search_profile_name="myHnswProfile"),
        SearchField(name="embeddingContent", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True, vector_search_dimensions=EMBEDDINGS_DIMENSIONS, vector_search_profile_name="myHnswProfile")
    ]

    # Configure the vector search configuration
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw",
                kind=VectorSearchAlgorithmKind.HNSW,
                parameters=HnswParameters(
                    m=4,
                    ef_construction=400,
                    ef_search=500,
                    metric=VectorSearchAlgorithmMetric.COSINE
                )
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw",
            )
        ]
    )

    # Semantic ranker configuration
    semantic_config = SemanticConfiguration(
        name="semantic-config",
        prioritized_fields=SemanticPrioritizedFields(
            title_field=SemanticField(field_name="title"),
            content_fields=[SemanticField(field_name="content")]
        )
    )

    # Create the semantic settings with the configuration
    semantic_search = SemanticSearch(configurations=[semantic_config])

    # Create the search index with the semantic settings
    index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_search=semantic_search)
    result = index_client.create_or_update_index(index)
    print(f"Index '{result.name}' created")

# Chunking Fixed tokens with LangChain
def chunk_text(title, text):
    text_splitter = TokenTextSplitter(
        chunk_size=MAX_TOKENS,
        chunk_overlap=OVERLAP_TOKENS
        )
    chunks = text_splitter.split_text(text)

    data = []
    for chunk in chunks:
        row = {'title': title, 'content': chunk}
        data.append(row)

    return data

# Index documents in the Azure AI Search index
# Index the batch in Azure AI Search index
def index_lote(batch_client, lote, i):
    try:
        print(f'Indexing until document {i+1}...')
        batch_client.upload_documents(documents=lote)
        print('Waiting 15 seconds...')
        time.sleep(15)
    except Exception as ex:
        print(ex)

# Index the contents or chunks
def index_documents(ai_search_endpoint, ai_search_credential, index_name, embedding_client, embedding_model_name, contents):
    print(f'Indexing {len(contents)} documents in {index_name} index...')
    # Create an index batch client
    batch_client = SearchIndexingBufferedSender(
                endpoint=ai_search_endpoint,
                index_name=index_name,
                credential=ai_search_credential
            )

    lote = []
    for i, content in enumerate(contents):  # Index the chunks using the file name as title
        #print('=================================================================')
        title = content['title']
        content = content['content']
        print(f"[{i + 1}]: title: {title}")
        #print(f"\t[{content}]")
        document = {
            "id": str(i),
            "title": title,
            "content": content,
            # Create embeddings with ADA-2
            "embeddingTitle": embedding_client.embeddings.create(input=cut_max_tokens(title), model=embedding_model_name).data[0].embedding,
            "embeddingContent": embedding_client.embeddings.create(input=cut_max_tokens(content), model=embedding_model_name).data[0].embedding,
        }
        # Add the document to the batch
        lote.append(document)
        # Index every 10 documents in the batch
        if (i + 1) % 10 == 0:
            # Upload documents
            print(f'INDEXING BATCH {i + 1}')
            index_lote(batch_client, lote, i)
            lote = []

    # Index the rest of documents after the last batch
    if len(lote) > 0:
        index_lote(batch_client, lote, i)

# Prepare AI Search client for testing
def get_ai_search_client(index_name):
    # Create an Azure AI Search client
    ai_search_client = SearchClient(endpoint=ai_search_config["ai_search_endpoint"],
                                    index_name=index_name,
                                    credential=AzureKeyCredential(ai_search_config["ai_search_apikey"]))
    return ai_search_client

## Index data from a database

### Query in a PostgreSQL database
- Requirements: pip install psycopg2

Configure the PostgreSQL connection in the .env file:
- PG_HOST=your-pg-host
- PG_PORT=your-pg-port (usually 5432)
- PG_USER=your-pg-user
- PG_PASSWORD=your-pg-password
- PG_DATABASE=your-pg-database-name

In [3]:
import pg8000

# Connection configuration
pg_conn = pg8000.connect(
    user=os.getenv('PG_USER'),
    password=os.getenv('PG_PASSWORD'),
    host=os.getenv('PG_HOST'),
    port=os.getenv('PG_PORT'),
    database=os.getenv('PG_DATABASE')
)

def query_pg(conn, sql):
    cursor = conn.cursor()
    cursor.execute(sql)
    results = cursor.fetchall()
    print("Results:", results)
    cursor.close()
    return results

#### Prepare the AI Search index
- create the index
- get the data querying the database
- chunk the content
- index the data
- test a query in AI Search index

In [ ]:
# Create the index
create_index(ai_search_config["ai_search_index_name_regs"])

In [ ]:
# Define the SQL query
sql = "SELECT departmentid, name, groupname from humanresources.department;"
# Query in PostgreSQL
results = query_pg(pg_conn, sql)
# Convertir los resultados a un formato JSON
rows = []
for row in results:
    rows.append({
        "id": row[0], # departmentid
        "title": row[1], # name
        "content": row[2], # groupname
    })
print(json.dumps(rows, indent=2))

In [ ]:
# Chunk the values of field 'content'
chunks = []
for i, row in enumerate(rows):
    # Create chunks
    chunks += chunk_text(row['title'], row['content'])
print(f'Number of chunks: {len(chunks)}')
print(f'Chunks: {json.dumps(chunks, indent=2)}')

In [ ]:
# Index content retrieved from the database (NO CHUNKING)
#index_documents(ai_search_config["ai_search_endpoint"],
#                ai_search_config["ai_search_credential"],
#                ai_search_config["ai_search_index_name_regs"],
#                openai_config["openai_client"],
#                openai_config["aoai_embedding_model"],
#                rows)

# Index content retrieved from the database (CHUNKING)
index_documents(ai_search_config["ai_search_endpoint"],
                ai_search_config["ai_search_credential"],
                ai_search_config["ai_search_index_name_regs"],
                openai_config["openai_client"],
                openai_config["aoai_embedding_model"],
                chunks)

In [ ]:
# Test a query
query = "R&D"
ai_search_client = get_ai_search_client(ai_search_config["ai_search_index_name_regs"])
results, num_results = semantic_hybrid_search(ai_search_client,
                                              openai_config["openai_client"],
                                              openai_config["aoai_embedding_model"],
                                              query=query,
                                              max_docs=10)
show_results(results, query)

### Query in a database by endpoint
- Requirements: pip install flask
- Before sending a query to SQLite install Flask with 'pip install Flask' and run the following command: ***python app.py***

Customization the sample:
- **query_sqlite_endpoint:**: it sends the SQL query to the database using the endpoint through the Flask web server. Copy and modify it, substituting the parameters to your data source using the REST API.

In [11]:
def query_sqlite_endpoint(sqlite_endpoint, sql, user, password):
    # Define the headers and payload
    headers = {
        'Content-Type': 'application/json'
    }
    payload = {
        'query': sql,
        'user': user,
        'password': password
    }

    # Make the request
    response = requests.post(sqlite_endpoint, json=payload, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        return data
    else:
        print("Error executing the query. Status code:", response.status_code)
        print("Response:", response.text)
        return None


#### Prepare the AI Search index
- create the index
- get the data querying the database
- chunk the content
- index the data
- test a query in AI Search index

In [ ]:
# Create the index
create_index(ai_search_config["ai_search_index_name_regs"])

In [ ]:
# Define the SQL query
sql = """SELECT p.Name, d.Description
FROM Product AS p
JOIN ProductDescription AS d
ON p.ProductID = d.ProductDescriptionID
"""
# Query in the SQLite endpoint
response = query_sqlite_endpoint(sqlite_endpoint, sql, sqlite_user, sqlite_password)

if response != None:
    # Prepare the data in json where the first field is the title and the second is the content
    rows = [None] * len(response)
    # Prepare the data in json where the first field is the title and the second is the content
    for i, row in enumerate(response):
        rows[i] = {
            'title': row[0],
            'content': row[1]
        }
    print(json.dumps(rows, indent=2))

In [ ]:
# Chunk the values of field 'content'
chunks = []
for i, row in enumerate(rows):
    # Create chunks
    chunks += chunk_text(row['title'], row['content'])
print(f'Number of chunks: {len(chunks)}')
print(f'Chunks: {json.dumps(chunks, indent=2)}')


In [ ]:
# Index content retrieved from the database (NO CHUNKING)
#index_documents(ai_search_config["ai_search_endpoint"],
#                ai_search_config["ai_search_credential"],
#                ai_search_config["ai_search_index_name_regs"],
#                openai_config["openai_client"],
#                openai_config["aoai_embedding_model"],
#                rows)

# Index content retrieved from the database (CHUNKING)
index_documents(ai_search_config["ai_search_endpoint"],
                ai_search_config["ai_search_credential"],
                ai_search_config["ai_search_index_name_regs"],
                openai_config["openai_client"],
                openai_config["aoai_embedding_model"],
                chunks)

In [ ]:
# Test a query
query = "pantalones cortos de hombre"
ai_search_client = get_ai_search_client(ai_search_config["ai_search_index_name_regs"])
results, num_results = semantic_hybrid_search(ai_search_client,
                                              openai_config["openai_client"],
                                              openai_config["aoai_embedding_model"],
                                              query=query,
                                              max_docs=10)
show_results(results, query)

## Index contents of files in a local folder

### Functions to convert documents to markdown, chunk and indexing the chunks
- convert_files_to_markdown: convert every file in a folder to markdown with Document Intelligence
- chunk_and_index_md_files: chunk every markdown file and index the chunks

In [13]:
# Process every PDF in a directory
def convert_files_to_markdown(input_dir, output_dir, extension):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        if filename.endswith(extension):
            file_path = os.path.join(input_dir, filename)
            convert_file_to_markdown(file_path, output_dir)

# Convert one document to MARKDOWN
def convert_file_to_markdown(file_path, output_dir):
    output_file_path = os.path.join(output_dir, os.path.splitext(os.path.basename(file_path))[0] + '.md')
    
    print(f'Converting {file_path} to {output_file_path} in markdown format...')
    try:
        # Read the temporal file
        with open(file_path, "rb") as pdf_file:
            pdf_content = pdf_file.read()

        # Convert to markdown with Document Intelligence
        poller = doc_intel_client.begin_analyze_document("prebuilt-layout",
                                                        body=pdf_content,
                                                        output_content_format=DocumentContentFormat.MARKDOWN,
                                                        content_type="application/octet-stream")
        result = poller.result()
        markdown = result['content']

        # Save the markdown to disk
        with open(output_file_path, "w", encoding="utf-8") as f:
            f.write(markdown)
        print(f"\tSaved file [{output_file_path}]")

    except Exception as ex:
        markdown = None
        print(ex)

    return markdown

# Chunk and index the markdown files
def chunk_and_index_md_files(input_dir, index_name):
    for filename in os.listdir(input_dir):
        if filename.endswith('.md'):
            file_path = os.path.join(input_dir, filename)
            print(f'Chunking {file_path} -----------------------------')
            # Read the md file
            with open(file_path, "r", encoding='utf-8') as pdf_file:
                text = pdf_file.read()
            chunks = chunk_text(filename, text)

            # Index the chunk
            index_documents(ai_search_config["ai_search_endpoint"],
                            ai_search_config["ai_search_credential"],
                            index_name,
                            openai_config["openai_client"],
                            openai_config["aoai_embedding_model"],
                            chunks)

### Prepare the AI Search index
- create the index
- convert PDF files to markdown
- chunk and index the chunks
- test a query in AI Search index

In [ ]:
# Create the index
create_index(index_name = ai_search_config["ai_search_index_name_docs"])

In [ ]:
# Convert PDF files to markdown
convert_files_to_markdown(input_dir='docs', output_dir='docs/markdown', extension='.pdf')

In [ ]:
# Chunk and index the markdown files
chunk_and_index_md_files(input_dir='docs/markdown', index_name=ai_search_config["ai_search_index_name_docs"])

In [ ]:
# Test a query
query = "healthcare plan"
ai_search_client = get_ai_search_client(ai_search_config["ai_search_index_name_regs"])
results, num_results = semantic_hybrid_search(ai_search_client,
                                              openai_config["openai_client"],
                                              openai_config["aoai_embedding_model"],
                                              query=query,
                                              max_docs=10)
show_results(results, query)

## Index contents of files in a blob storage

### Functions to download the documents from the blob storage, convert them to markdown, chunk and indexing the chunks
- download_files_in_blob: convert every PDF in a folder to markdown with Document Intelligence
- convert_file(s)_to_markdown (defined in "Index contents of files in a local folder"): convert every files to markdown
- chunk_and_index_md_files (defined in "Index contents of files in a local folder"): chunk every markdown file and index the chunks

In [10]:
from azure.storage.blob import BlobServiceClient
load_dotenv(find_dotenv(), override=True)
connection_string = os.getenv("BLOB_CONNECTION_STRING")

# Process every PDF in a blob storage container
def download_files_in_blob(container_name, output_dir, extension):
    os.makedirs(output_dir, exist_ok=True)
    # Crear un BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)

    # Obtener el cliente del contenedor
    container_client = blob_service_client.get_container_client(container_name)

    # Listar todos los blobs en el contenedor
    blob_list = container_client.list_blobs()

    # Descargar al directorio local, procesarlo y borrarlo de local
    for blob in blob_list:
        blob_client = container_client.get_blob_client(blob.name)
        if blob.name.endswith(extension):
            file_path = os.path.join(output_dir, blob.name)
            print(f"Downloading {blob.name} to {file_path}...")
            # Descargar el blob
            with open(file_path, "wb") as download_file:
                download_file.write(blob_client.download_blob().readall())

### Prepare the AI Search index
- create the index
- convert PDF files to markdown
- chunk and index the chunks
- test a query in AI Search index

In [ ]:
# Create the index
create_index(index_name="rag-index-blob")

In [ ]:
# Download PDF files from blob storage container
download_files_in_blob(container_name='test', output_dir='blob_downloads', extension='pdf')

In [ ]:
# Convert PDF files to markdown
convert_files_to_markdown(input_dir='blob_downloads', output_dir='blob_downloads_md', extension='.pdf')

In [ ]:
# Chunk and index the markdown files
chunk_and_index_md_files(input_dir='blob_downloads_md', index_name="rag-index-blob")

In [ ]:
# Test a query
query = "Werden eine Daten an ein anderes Unternehmen weitergegeben?"
ai_search_client = get_ai_search_client(index_name="rag-index-blob")
results, num_results = semantic_hybrid_search(ai_search_client=ai_search_client,
                                              openai_client=openai_config["openai_client"],
                                              aoai_embedding_model=openai_config["aoai_embedding_model"],
                                              query=query,
                                              max_docs=10)


show_results(results, query)